In [1]:
import pandas as pd
import numpy as np

Сохраняем необходимые данные (чтобы не качать при каждом перезапуске ноутбука).

In [2]:
# cкачиваем все данные из user_data
user_data = pd.read_sql(
    "SELECT * FROM public.user_data",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем user_data в файл
user_data.to_csv('user_data.csv', index=False)

In [3]:
# скачиваем среднее по target по каждому user_id из feed_data
user_stat = pd.read_sql(
    "SELECT user_id, AVG(target) AS user_stat FROM public.feed_data GROUP BY user_id",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем user_data в файл
user_stat.to_csv('user_stat.csv', index=False)

In [4]:
# cкачиваем все данные из post_text
post_text = pd.read_sql(
    "SELECT * FROM public.post_text_df",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем post_text в файл
post_text.to_csv('post_text.csv', index=False)

In [5]:
# скачиваем среднее по target по каждому post_id из feed_data
post_stat = pd.read_sql(
    "SELECT post_id, AVG(target) AS post_stat FROM public.feed_data GROUP BY post_id",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем post_text в файл
post_stat.to_csv('post_stat.csv', index=False)

In [6]:
# скачиваем последние 100 тысяч записей в feed_data
feed_data = pd.read_sql(
    "SELECT * FROM public.feed_data WHERE action = 'view' ORDER BY timestamp DESC LIMIT 100000",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем feed_data в файл
feed_data.to_csv('feed_data.csv', index=False)

Загружаем данные из ранее сохранённых файлов

In [7]:
post_text = pd.read_csv('post_text.csv')

In [8]:
post_stat = pd.read_csv('post_stat.csv')

In [9]:
user_data = pd.read_csv('user_data.csv')

In [10]:
user_stat = pd.read_csv('user_stat.csv')

In [11]:
feed_data = pd.read_csv('feed_data.csv')
feed_data.sort_values('timestamp', inplace=True, ignore_index=True)

В таблице feed_data есть колонка 'target', она будет являться целевой в нашем обучении.

0 - пост не получил like от пользователя, которому был показан.

1 - пост получил like.

При помощи метода TF-IDF в таблице post_text преобразуем текстовую колонку 'text' в векторный вид.
Из получившейся матрицы возьмём построчные среднее, максимум и сумму.

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

post_text_copy = post_text.copy()

vectorizer = TfidfVectorizer()
F = vectorizer.fit_transform(post_text['text'])
post_text_copy['text_mean'] = F.toarray().mean(axis=1)
post_text_copy['text_max'] = F.toarray().max(axis=1)
post_text_copy['text_sum'] = F.toarray().sum(axis=1)
post_text_copy = post_text_copy.drop('text', axis=1)

При помощи метода главных компонентов понизим размерность полученной выше матрицы векторов текстов и добавим как признаки.

In [13]:
from sklearn.decomposition import PCA

n = 5
columns = [f"text_{i}th_cluster" for i in range(1, n+1)]

pca = PCA(n_components=n, random_state=1)
pca_decomp = pca.fit_transform(F.toarray())

pca_decomp = pd.DataFrame(pca_decomp, columns=columns)

post_text_copy = pd.concat((post_text_copy, pca_decomp), axis=1)

Соединяем таблицы по колонкам post_id и user_id.

In [14]:
df = pd.merge(feed_data,
              post_text_copy,
              on='post_id',
              how='left')

In [15]:
df = pd.merge(df,
              post_stat,
              on='post_id',
              how='left')

In [16]:
df = pd.merge(df,
              user_data,
              on='user_id',
              how='left')

In [17]:
df = pd.merge(df,
              user_stat,
              on='user_id',
              how='left')

Из колонки timestamp выделяем дополнительные признаки: год, месяц, день, час, минута, секунда. А исходную колонку удаляем.

In [18]:
from datetime import datetime as dt

df['timestamp'] = df['timestamp'].astype('datetime64')

df['year'] = df['timestamp'].dt.year # .astype('object')
df['month'] = df['timestamp'].dt.month # .astype('object')
df['day'] = df['timestamp'].dt.day # .astype('object')
df['hour'] = df['timestamp'].dt.hour # .astype('object')
df['minute'] = df['timestamp'].dt.minute #.astype('object')
df['second'] = df['timestamp'].dt.second #.astype('object')

df = df.drop('timestamp', axis=1)

Делим сгруппированый набор данных на признаки "X" и целевую колонку "Y".

In [19]:
X = df.drop(['user_id', 'post_id', 'action', 'target'], axis=1)
y = df['target']

Делим колонки в таблице признаков на числовые и категориальные, последние в свою очередь делим на признаки, которые будут обрабатываться методом One-Hot или кодирования по таргету.

In [20]:
numeric_columns = list(X.select_dtypes(exclude='object').columns)
categorical_columns = list(X.select_dtypes(include='object').columns)

columns_ohe = [x for x in categorical_columns if X[x].nunique() < 5]
columns_mte = [x for x in categorical_columns if X[x].nunique() >= 5]

numeric_columns_ind = [list(X.columns).index(col) for col in numeric_columns]
columns_ohe_ind = [list(X.columns).index(col) for col in columns_ohe]
columns_mte_ind = [list(X.columns).index(col) for col in columns_mte]

Создаём трансформер, который будет преобразовывать данные признаков следующим образом: числовые признаки стандартизирует, а к категориальным применит метод One-Hot и TargetEncoder.

In [21]:
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.feature_extraction.text import TfidfVectorizer

t = [('StandardScaler', StandardScaler(), numeric_columns_ind),
     ('OneHotEncoder', OneHotEncoder(), columns_ohe_ind),
     ('MeanTargetEncoder', TargetEncoder(), columns_mte_ind)]

columns_transformer = ColumnTransformer(transformers=t)

D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Делим данные на тренеровочные и тестовые.

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1)

При помощи Pipeline проводим обучение модели CatBoostClassifier. Параметры модели были подобранны вручную.

In [23]:
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier

In [24]:
%%time

pipe = Pipeline([('columns_transformer', columns_transformer),
                 ('decision_tree', CatBoostClassifier(iterations = 1500,
                                                      loss_function = 'MultiClass',
                                                      early_stopping_rounds=10,
                                                      thread_count = 100,
                                                      random_state=1))])
pipe.fit(X_train, y_train)

D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Learning rate set to 0.07024
0:	learn: 0.6602675	total: 173ms	remaining: 4m 19s
1:	learn: 0.6320620	total: 187ms	remaining: 2m 19s
2:	learn: 0.6073345	total: 200ms	remaining: 1m 39s
3:	learn: 0.5854868	total: 213ms	remaining: 1m 19s
4:	learn: 0.5663001	total: 226ms	remaining: 1m 7s
5:	learn: 0.5495004	total: 239ms	remaining: 59.4s
6:	learn: 0.5345503	total: 251ms	remaining: 53.5s
7:	learn: 0.5211007	total: 264ms	remaining: 49.3s
8:	learn: 0.5091303	total: 277ms	remaining: 46s
9:	learn: 0.4988403	total: 291ms	remaining: 43.3s
10:	learn: 0.4897087	total: 304ms	remaining: 41.1s
11:	learn: 0.4814385	total: 318ms	remaining: 39.5s
12:	learn: 0.4741697	total: 330ms	remaining: 37.8s
13:	learn: 0.4676460	total: 343ms	remaining: 36.4s
14:	learn: 0.4619862	total: 356ms	remaining: 35.3s
15:	learn: 0.4567310	total: 368ms	remaining: 34.1s
16:	learn: 0.4520642	total: 380ms	remaining: 33.2s
17:	learn: 0.4478901	total: 394ms	remaining: 32.4s
18:	learn: 0.4442358	total: 407ms	remaining: 31.7s
19:	learn:

172:	learn: 0.3890337	total: 2.36s	remaining: 18.1s
173:	learn: 0.3889670	total: 2.37s	remaining: 18.1s
174:	learn: 0.3886829	total: 2.38s	remaining: 18.1s
175:	learn: 0.3885645	total: 2.4s	remaining: 18s
176:	learn: 0.3883113	total: 2.41s	remaining: 18s
177:	learn: 0.3882441	total: 2.42s	remaining: 18s
178:	learn: 0.3881772	total: 2.43s	remaining: 17.9s
179:	learn: 0.3881164	total: 2.44s	remaining: 17.9s
180:	learn: 0.3878660	total: 2.46s	remaining: 17.9s
181:	learn: 0.3878124	total: 2.47s	remaining: 17.9s
182:	learn: 0.3877304	total: 2.48s	remaining: 17.8s
183:	learn: 0.3875252	total: 2.49s	remaining: 17.8s
184:	learn: 0.3874821	total: 2.5s	remaining: 17.8s
185:	learn: 0.3873453	total: 2.52s	remaining: 17.8s
186:	learn: 0.3871122	total: 2.53s	remaining: 17.7s
187:	learn: 0.3869503	total: 2.54s	remaining: 17.7s
188:	learn: 0.3869068	total: 2.55s	remaining: 17.7s
189:	learn: 0.3865511	total: 2.56s	remaining: 17.7s
190:	learn: 0.3860057	total: 2.58s	remaining: 17.7s
191:	learn: 0.385785

345:	learn: 0.3688320	total: 4.48s	remaining: 14.9s
346:	learn: 0.3686426	total: 4.49s	remaining: 14.9s
347:	learn: 0.3686030	total: 4.5s	remaining: 14.9s
348:	learn: 0.3685729	total: 4.51s	remaining: 14.9s
349:	learn: 0.3685363	total: 4.53s	remaining: 14.9s
350:	learn: 0.3684749	total: 4.54s	remaining: 14.9s
351:	learn: 0.3684369	total: 4.55s	remaining: 14.8s
352:	learn: 0.3683264	total: 4.56s	remaining: 14.8s
353:	learn: 0.3682134	total: 4.58s	remaining: 14.8s
354:	learn: 0.3681645	total: 4.59s	remaining: 14.8s
355:	learn: 0.3681212	total: 4.6s	remaining: 14.8s
356:	learn: 0.3680826	total: 4.61s	remaining: 14.8s
357:	learn: 0.3680157	total: 4.62s	remaining: 14.7s
358:	learn: 0.3679412	total: 4.64s	remaining: 14.7s
359:	learn: 0.3678608	total: 4.65s	remaining: 14.7s
360:	learn: 0.3678238	total: 4.66s	remaining: 14.7s
361:	learn: 0.3677727	total: 4.67s	remaining: 14.7s
362:	learn: 0.3676915	total: 4.69s	remaining: 14.7s
363:	learn: 0.3673913	total: 4.7s	remaining: 14.7s
364:	learn: 0.3

508:	learn: 0.3560809	total: 6.47s	remaining: 12.6s
509:	learn: 0.3559428	total: 6.48s	remaining: 12.6s
510:	learn: 0.3556884	total: 6.49s	remaining: 12.6s
511:	learn: 0.3556185	total: 6.51s	remaining: 12.6s
512:	learn: 0.3555839	total: 6.52s	remaining: 12.5s
513:	learn: 0.3555394	total: 6.53s	remaining: 12.5s
514:	learn: 0.3554970	total: 6.54s	remaining: 12.5s
515:	learn: 0.3554560	total: 6.55s	remaining: 12.5s
516:	learn: 0.3554242	total: 6.57s	remaining: 12.5s
517:	learn: 0.3553396	total: 6.58s	remaining: 12.5s
518:	learn: 0.3552937	total: 6.59s	remaining: 12.5s
519:	learn: 0.3552101	total: 6.6s	remaining: 12.4s
520:	learn: 0.3550414	total: 6.62s	remaining: 12.4s
521:	learn: 0.3549856	total: 6.63s	remaining: 12.4s
522:	learn: 0.3549353	total: 6.64s	remaining: 12.4s
523:	learn: 0.3548384	total: 6.65s	remaining: 12.4s
524:	learn: 0.3548044	total: 6.67s	remaining: 12.4s
525:	learn: 0.3547657	total: 6.68s	remaining: 12.4s
526:	learn: 0.3547245	total: 6.69s	remaining: 12.3s
527:	learn: 0

...

1005:	learn: 0.3287544	total: 12.5s	remaining: 6.16s
1006:	learn: 0.3287316	total: 12.6s	remaining: 6.15s
1007:	learn: 0.3287068	total: 12.6s	remaining: 6.13s
1008:	learn: 0.3286703	total: 12.6s	remaining: 6.12s
1009:	learn: 0.3286468	total: 12.6s	remaining: 6.11s
1010:	learn: 0.3286167	total: 12.6s	remaining: 6.09s
1011:	learn: 0.3285964	total: 12.6s	remaining: 6.08s
1012:	learn: 0.3285530	total: 12.6s	remaining: 6.07s
1013:	learn: 0.3285246	total: 12.6s	remaining: 6.06s
1014:	learn: 0.3284917	total: 12.7s	remaining: 6.05s
1015:	learn: 0.3284668	total: 12.7s	remaining: 6.04s
1016:	learn: 0.3284225	total: 12.7s	remaining: 6.03s
1017:	learn: 0.3283937	total: 12.7s	remaining: 6.02s
1018:	learn: 0.3283223	total: 12.7s	remaining: 6s
1019:	learn: 0.3283004	total: 12.7s	remaining: 5.99s
1020:	learn: 0.3282660	total: 12.7s	remaining: 5.98s
1021:	learn: 0.3282524	total: 12.8s	remaining: 5.97s
1022:	learn: 0.3281948	total: 12.8s	remaining: 5.95s
1023:	learn: 0.3281592	total: 12.8s	remaining: 5.

1165:	learn: 0.3219632	total: 14.5s	remaining: 4.16s
1166:	learn: 0.3219446	total: 14.5s	remaining: 4.15s
1167:	learn: 0.3219192	total: 14.5s	remaining: 4.13s
1168:	learn: 0.3218945	total: 14.6s	remaining: 4.12s
1169:	learn: 0.3218662	total: 14.6s	remaining: 4.11s
1170:	learn: 0.3218423	total: 14.6s	remaining: 4.1s
1171:	learn: 0.3217991	total: 14.6s	remaining: 4.08s
1172:	learn: 0.3216428	total: 14.6s	remaining: 4.07s
1173:	learn: 0.3215978	total: 14.6s	remaining: 4.06s
1174:	learn: 0.3215865	total: 14.6s	remaining: 4.05s
1175:	learn: 0.3215468	total: 14.6s	remaining: 4.04s
1176:	learn: 0.3215182	total: 14.7s	remaining: 4.02s
1177:	learn: 0.3214754	total: 14.7s	remaining: 4.01s
1178:	learn: 0.3214419	total: 14.7s	remaining: 4s
1179:	learn: 0.3214322	total: 14.7s	remaining: 3.99s
1180:	learn: 0.3214072	total: 14.7s	remaining: 3.97s
1181:	learn: 0.3213736	total: 14.7s	remaining: 3.96s
1182:	learn: 0.3213515	total: 14.7s	remaining: 3.95s
1183:	learn: 0.3213097	total: 14.8s	remaining: 3.9

1323:	learn: 0.3158123	total: 16.4s	remaining: 2.19s
1324:	learn: 0.3157812	total: 16.5s	remaining: 2.17s
1325:	learn: 0.3157524	total: 16.5s	remaining: 2.16s
1326:	learn: 0.3157306	total: 16.5s	remaining: 2.15s
1327:	learn: 0.3157012	total: 16.5s	remaining: 2.14s
1328:	learn: 0.3156718	total: 16.5s	remaining: 2.12s
1329:	learn: 0.3156453	total: 16.5s	remaining: 2.11s
1330:	learn: 0.3156054	total: 16.6s	remaining: 2.1s
1331:	learn: 0.3155743	total: 16.6s	remaining: 2.09s
1332:	learn: 0.3155402	total: 16.6s	remaining: 2.08s
1333:	learn: 0.3155106	total: 16.6s	remaining: 2.06s
1334:	learn: 0.3154927	total: 16.6s	remaining: 2.05s
1335:	learn: 0.3154552	total: 16.6s	remaining: 2.04s
1336:	learn: 0.3154343	total: 16.6s	remaining: 2.03s
1337:	learn: 0.3154133	total: 16.6s	remaining: 2.01s
1338:	learn: 0.3152607	total: 16.6s	remaining: 2s
1339:	learn: 0.3152231	total: 16.7s	remaining: 1.99s
1340:	learn: 0.3151845	total: 16.7s	remaining: 1.98s
1341:	learn: 0.3151584	total: 16.7s	remaining: 1.9

1492:	learn: 0.3099414	total: 18.6s	remaining: 87.2ms
1493:	learn: 0.3099234	total: 18.6s	remaining: 74.8ms
1494:	learn: 0.3099028	total: 18.6s	remaining: 62.3ms
1495:	learn: 0.3097995	total: 18.6s	remaining: 49.8ms
1496:	learn: 0.3097782	total: 18.7s	remaining: 37.4ms
1497:	learn: 0.3097488	total: 18.7s	remaining: 24.9ms
1498:	learn: 0.3097199	total: 18.7s	remaining: 12.5ms
1499:	learn: 0.3096829	total: 18.7s	remaining: 0us
CPU times: total: 3min 6s
Wall time: 20.1 s


Pipeline(steps=[('columns_transformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 14, 17, 18, 19, 20,
                                                   21, 22, 23]),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(), [15, 16]),
                                                 ('MeanTargetEncoder',
                                                  TargetEncoder(),
                                                  [0, 12, 13])])),
                ('decision_tree',
                 <catboost.core.CatBoostClassifier object at 0x0000021CFFADD8E0>)])

Сохраняем обученую модель.

In [25]:
import pickle

filename = 'cat_model.pkl'
pickle.dump(pipe, open(filename, 'wb'))

Рассчитаем ROC-AUC для обученной модели на тестовых данных.

In [26]:
from sklearn.metrics import roc_auc_score, accuracy_score

print(f'ROC-AUC {roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1])}')

ROC-AUC 0.7810780728524609


# Далее делаем пайплайн расчёта предсказаний
Для для рандомного пользователя и рандомного времени получим предсказания от ранее обученной модели, чтобы проверить её работоспособность.

In [28]:
note = post_text.copy()

vectorizer = TfidfVectorizer()
F = vectorizer.fit_transform(note['text'])
note['text_mean'] = F.toarray().mean(axis=1)
note['text_max'] = F.toarray().max(axis=1)
note['text_sum'] = F.toarray().sum(axis=1)
note = note.drop('text', axis=1)

In [29]:
columns = [f"text_{i}th_cluster" for i in range(1, n+1)]

pca = PCA(n_components=n, random_state=1)
pca_decomp = pca.fit_transform(F.toarray())

pca_decomp = pd.DataFrame(pca_decomp, columns=columns)

note = pd.concat((note, pca_decomp), axis=1)

In [30]:
note = pd.merge(note,
                post_stat,
                on='post_id',
                how='left')

Сохраняем таблицу note на сервер и сразу скачиваем её для проверки, что всё сохранилось как и задумывалось.

In [31]:
note.to_sql(
    "note_shaverdin",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml",
    schema="public",
    if_exists='replace',
    index=False
)

note_shaverdin = pd.read_sql(
    """SELECT * FROM note_shaverdin""",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
note_shaverdin.head(1)

,post_id,topic,text_mean,text_max,text_sum,text_1th_cluster,text_2th_cluster,text_3th_cluster,text_4th_cluster,text_5th_cluster,post_stat
0,1,business,0.000204,0.439495,10.593563,-0.118109,-0.202089,-0.092041,-0.111281,-0.090189,0.111436


In [32]:
users = pd.merge(user_data,
                 user_stat,
                 on='user_id',
                 how='left')

Сохраняем таблицу users на сервер и сразу скачиваем её для проверки, что всё сохранилось как и задумывалось.

In [33]:
users.to_sql(
    "users_shaverdin",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml",
    schema="public",
    if_exists='replace',
    index=False
)

users_shaverdin = pd.read_sql(
    """SELECT * FROM users_shaverdin""",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
users_shaverdin.head(1)

,user_id,gender,age,country,city,exp_group,os,source,user_stat
0,200,1,34,Russia,Degtyarsk,3,Android,ads,0.107232


In [34]:
id = 345
time = dt(year=2021, month=10, day=30, hour=14)
limit = 10

In [35]:
def add_user_time(note, pers, time):
    note['gender'] = pers[0]
    note['age'] = pers[1]
    note['country'] = pers[2]
    note['city'] = pers[3]
    note['exp_group'] = pers[4]
    note['os'] = pers[5]
    note['source'] = pers[6]
    note['user_stat'] = pers[7]
    
    note['year'] = time.year
    note['month'] = time.month
    note['day'] = time.day
    note['hour'] = time.hour
    note['minute'] = time.minute
    note['second'] = time.second
    
    return note

In [36]:
%%time

pers = users[users['user_id']==id].drop('user_id', axis=1).iloc[0]

pred = add_user_time(note, pers, time).drop('post_id', axis=1)

predict = pd.DataFrame(post_text['post_id'])

predict['predict'] = pipe.predict_proba(pred)[:,1]

predict.sort_values('predict', ascending=False, inplace=True, ignore_index=True)

posts = predict.head(limit)['post_id'].values

posts = post_text[post_text['post_id'].isin(posts)]

posts

CPU times: total: 46.9 ms
Wall time: 105 ms


,post_id,text,topic
1865,1517,Campbell lifts lid on United feud\n\nArsenals ...,sport
1892,2773,What have you been listening to during #COVID1...,covid
2053,1688,"Athens memories soar above lows\n\nWell, its g...",sport
2060,1694,2004: An Irish Athletics Year\n\n2004 wont be ...,sport
2181,1797,Evertons Weir cools Euro hopes\n\nEverton defe...,sport
2540,2062,Losing yourself in online gaming\n\nOnline rol...,tech
2649,2980,Big NO on this — lives are more important than...,covid
2789,3033,Did #RobertTrump die from #COVID19 ?,covid
4070,4251,"Its about an embezzler, Peter Ustinov, who inf...",movie
4644,4833,One day a red alley cat is fed up of being kic...,movie
